In [11]:
from qiskit import QuantumCircuit
from qiskit.quantum_info import partial_trace, Statevector
from qiskit.circuit.library import ZGate, XGate
from qiskit import execute, Aer
from qiskit.compiler import transpile
from qiskit.tools.visualization import plot_histogram
from matplotlib import pyplot as plt
from matplotlib import colors
import numpy as np
import gc
import csv
from pylab import *

In [3]:
def createSFblock(N, j):
    
    circuit = QuantumCircuit(N)
    
    state_decimal = format(j, '0'+str(N)+'b')[::-1] # inverse order due to Qiskit order: |q1q0>
    
    for qubit in range(N):
        if state_decimal[qubit] == '0':
            circuit.x(qubit)
    
    multiConrolledZ = ZGate().control(num_ctrl_qubits=N-1)
    circuit.append(multiConrolledZ, range(N))
    
    for qubit in range(N):
        if state_decimal[qubit] == '0':
            circuit.x(qubit)
    
    return circuit


In [4]:
def U_i(N, ki, circuit):
    
    # create equal superposition
    circuit.h(range(N))
    
    # apply SF blocks according to ki information
    vector_i = format(ki, '0'+str(2**N)+'b')
    for ij in range(len(vector_i)):
        if vector_i[ij] == '1': # n_j = 1, equivalent of having -1 in vector i
            circuit.compose(createSFblock(N, ij), inplace=True)  # the state i is the one that must have the phase of -1

def U_w(N, kw, circuit):
    
    # apply SF blocks according to kw information
    vector_w = format(kw, '0'+str(2**N)+'b')
    for wj in range(len(vector_w)):
        if vector_w[wj] == '1': # n_j = 1, equivalent of having -1 in vector w
            circuit.compose(createSFblock(N, wj), inplace=True)  # the state w is the one that must have the phase of -1
    
    # apply Hadamards
    circuit.h(range(N))
    
    # apply NOTs
    circuit.x(range(N))

In [5]:
def perceptron_brute_force(N, ki, kw, draw=False):
    
    # create circuit, N qubits + ancilla
    circuit = QuantumCircuit(N + 1, 1)
    
    # apply U_i
    U_i(N, ki, circuit)
    
    # apply U_w
    U_w(N, kw, circuit)
    
    # apply C^N X
    circuit.mcx(control_qubits=[i for i in range(N)], target_qubit=N)
    
    # measure the ancilla qubit
    circuit.measure(N, 0)
    
    # draw circuit
    if draw == True:
        display(circuit.draw('mpl'))
    
    return circuit

In [9]:
%%time

# Calculate for every value for ki and kw

N = 3

for kw in range(2**(2**N)):
    
    print("\rworking with kw = {}/{}".format(kw, 2**(2**N)), end="")
    
    circuits_res = []
    
    for ki in range(2**(2**N)):
        perceptron_circuit = perceptron_brute_force(N, ki, kw)
        perceptron_circuit.remove_final_measurements()
        full_state = Statevector(perceptron_circuit)
        partial_density_matrix_ancilla = partial_trace(full_state, range(N))
        circuits_res.append(
            {0: abs(partial_density_matrix_ancilla.data[0][0]),
             1: abs(partial_density_matrix_ancilla.data[1][1])})
    
    with open("res_exact/results_fake_belem_exact_Ui_Uw_N3_kw_{}.txt".format(kw), "w") as file:
        file.write(str(circuits_res))
    
    del circuits_res
    gc.collect()
    

working with kw = 255/256CPU times: total: 1h 34min 58s
Wall time: 1h 35min 35s


In [12]:
%%time
# load results from generated files and save them in matrix form in a new file

N = 3

with open("res_exact/results_fake_belem_exact_Ui_Uw_N3_all.txt", "w", newline="") as f:
    
    writer = csv.writer(f, delimiter=' ')
    
    for kw in range(2**(2**N)):

        print("\rreading for kw = {}/{}".format(kw, 2**(2**N)), end="")

        with open("res_exact/results_fake_belem_exact_Ui_Uw_N3_kw_{}.txt".format(kw), "r") as file:
            results_for_kw = eval(file.readline())

        results_for_state1 = []
        for ki in range(2**(2**N)):
            results_for_state1.append(results_for_kw[ki][1])
        
        writer.writerow(results_for_state1)

        del results_for_kw
        del results_for_state1
        gc.collect()
    

reading for kw = 255/256CPU times: total: 20 s
Wall time: 23.8 s
